# Class-Dependent Service Analysis

This example demonstrates **class-dependent service allocation**, which is an advanced load-dependent modeling technique.

## Concept:
- **Class1 jobs**: Can utilize up to 2 servers based on their population at the queue
- **Class2 jobs**: Always receive single-server treatment regardless of population
- **Service allocation**: min(Class1_population, 2) servers allocated to Class1

## Key Features:

1. **Asymmetric Service**: Different job classes receive different levels of service capacity
2. **Dynamic Allocation**: Server allocation changes based on specific class populations
3. **Resource Prioritization**: Class1 gets priority access to multiple servers

## Expected Behavior:
- Class1 should have better performance (lower response times) due to multi-server access
- Class2 performance is limited by single-server constraint
- Total system performance depends on the interaction between both classes

In [ ]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.mva.*
import jline.solvers.nc.*
import jline.solvers.ctmc.*
import jline.util.matrix.*

In [ ]:
// Model parameters
val N = 16  // number of jobs in Class1
val c = 2   // number of servers available for Class1

// Create class-dependent model
println("=== Class-Dependent Service Model ===")
val cdmodel = Network("cdmodel")
val node1 = Delay(cdmodel, "Delay")
val node2 = Queue(cdmodel, "Queue1", SchedStrategy.PS)

// Create job classes
val jobclass1 = ClosedClass(cdmodel, "Class1", N, node1, 0)
val jobclass2 = ClosedClass(cdmodel, "Class2", N / 2, node1, 0)  // N/2 = 8 jobs

// Set service times
node1.setService(jobclass1, Exp.fitMean(1.0))  // Class1: mean = 1.0
node1.setService(jobclass2, Exp.fitMean(2.0))  // Class2: mean = 2.0
node2.setService(jobclass1, Exp.fitMean(1.5))  // Class1: mean = 1.5
node2.setService(jobclass2, Exp.fitMean(2.5))  // Class2: mean = 2.5

In [ ]:
// Create routing matrix
val P = cdmodel.initRoutingMatrix()
P.set(jobclass1, jobclass1, node1, node2, 1.0)
P.set(jobclass1, jobclass1, node2, node1, 1.0)
P.set(jobclass2, jobclass2, node1, node2, 1.0)
P.set(jobclass2, jobclass2, node2, node1, 1.0)
cdmodel.link(P)

In [ ]:
// Set class dependence function
// In MATLAB: @(ni) min(ni(1),c) where ni(1) is Class1 population at the station
// The function should return min(Class1_population, c) servers for Class1 jobs

println("Setting class dependence...")
try {
    // Create load dependence as approximation
    val totalJobs = N + N / 2
    val alpha = DoubleArray(totalJobs + 1) { i ->
        kotlin.math.min(i.toDouble(), c.toDouble())
    }
    
    node2.setLoadDependence(alpha.toList())
    println("Using load dependence: ${alpha.toList()}")
} catch (e: Exception) {
    println("Load dependence setup failed: ${e.message}")
}

In [ ]:
// Solve with MVA using QD (queue decomposition) method
println("=== MVA Solver (QD) ===")
try {
    val solverMvaQd = MVA(cdmodel, "method", "qd")
    val cdAvgTableCD = solverMvaQd.avgTable
    println("MVA QD Results:")
    cdAvgTableCD.print()
} catch (e: Exception) {
    println("MVA QD solver failed: ${e.message}")
    
    // Try with exact MVA as fallback
    println("\nTrying MVA Exact method as fallback...")
    try {
        val solverMvaExact = MVA(cdmodel, "method", "exact")
        val cdAvgTableExact = solverMvaExact.avgTable
        println("MVA Exact Results:")
        cdAvgTableExact.print()
    } catch (e2: Exception) {
        println("MVA Exact also failed: ${e2.message}")
    }
}

In [ ]:
// Try NC solver as another option
println("=== NC Solver ===")
try {
    val solverNc = NC(cdmodel)
    val cdAvgTableNC = solverNc.avgTable
    println("NC Results:")
    cdAvgTableNC.print()
} catch (e: Exception) {
    println("NC solver failed: ${e.message}")
}

In [ ]:
// Solve with CTMC (exact solution)
println("=== CTMC Solver (Exact) ===")
try {
    val solverCtmc = CTMC(cdmodel)
    val cdAvgTableCTMC = solverCtmc.avgTable
    println("CTMC Results:")
    cdAvgTableCTMC.print()
} catch (e: Exception) {
    println("CTMC solver not available or failed: ${e.message}")
}